In [1]:
import pandas as pd
import numpy as np

# Vocabulario

El primer paso dentro del procesmaiento de lenguage natural implica la construcción de un vocabulario. El objetivo es convertir secuencias de texto en unidades *minimas* de escritura con *significado* (tokens). Para este curso usaremos palabras, $n$-gramas y $q$-gramas. Sin embargo, las técnicas de preprocesamiento pueden ser extendidas facilmente a equaciones, emoticones o cualquier otra unidad de escritura.   

Como lidiamos con lenguaje escrito, la obtención de los tokens requiere manipulación de cadenas de caracteres. Se requiere identificar puntuación, signo diacríticos (dieresis, acententos), en el caso del Inglés es posible que se desee dividir las contracciones (You're -> you are). Una vez construido el vocabulario mediante la **tokenización** de todos los documentos, puede  realizarse una reducción del mismo mediante un proceso de **lematización** o **steamming**. Ya con el vocabulario es posible construir una representación vectorial. 

Note que la tokenización podría ser a nível de sufijos/prefijos, silabas o incluso letras, pero por el momento solo lidiaremos con palabras. También es posible construir unidades formadas por 2,3 o $n$ palabras, a esto tokens se les conoce como $n$-gramas y nos permitene incluir conceptos que de más de una unidad por ejemplo en inglés *ice cream*.

## Tokenizador

Es proceso de tokenizado es un proceso de segmentación de *documentos*. Donde la segmentación es dividir el texto (información no estructurada) en unidades más pequeñas que pueden ser contabilizadas de forma discreta. El resultado de la contabilización de las ocurrencias de cada término puede ser utilizada directamente como un representación vectorial del documento. Con lo cual se transforma una entrada de información no estructurada en información estructurada que puedes ser utilizada por algoritmos de aprendizaje automático. La aplicación más común de este tipo de vectores (**bag of words**) para recuperación de documentos o búsqueda. 

El tokenizador más simple consiste en utilizar el espacio en blanco como delimitador para definir los tokens en terminos de palabras. En python se puede hacer como sigue:

In [35]:
doc="Pepe pecas pica papas con un pico, con un pico pepe pecas pica papas."
doc.split()

['Pepe',
 'pecas',
 'pica',
 'papas',
 'con',
 'un',
 'pico,',
 'con',
 'un',
 'pico',
 'pepe',
 'pecas',
 'pica',
 'papas.']

Utilizando el  método *split* pareciera se tiene un tokenizador medianamente bueno, sin embargo hay al menos dos situaciones no deseadas la primera los tokes *pico,*  y *papas.* esos tokens incluyen signos de puntuación, además los tokens como *Pepe* y *pepe* serán considerados como elementos distintos. Un tokenizador mas soifisticado debería separar los tokens de la puntuación, por ahora dejaremos este cuestión pero la retomaremos más adelante. Una forma básica de obtener una representación númerica de una secuencia de texto es mediante una representación binaria de cada token que existe en el vocabulario, está representación es conocida como **one-hot vectors**. Cada sentencia es representada como una lista de one-hot vectors por ejemplo:

In [41]:
## obtenemos el vocabulario a partir
doc1="Pepe pecas pica papas con un pico"
vocabulario=str.split(doc1) #Utilizamos el tokenizador
## se ordena
vocabulario.sort()
n=len(vocabulario)
print(f"El tamaño del vocabulario: {n}")

El tamaño del vocabulario: 7


In [42]:
vocabulario

['Pepe', 'con', 'papas', 'pecas', 'pica', 'pico', 'un']

Cada one-hot vector binaria será del tamaño del vocabulario y tendrá solo un 1 en la posición que corresponde a la palabra que representa.  

In [44]:
## Creamos la tabla de one-hot vectors 
one_hot_vectors=np.zeros((n,n)) 
## generamos la representación vectorial para nuesta frase de ejemplo
for i,w in enumerate(doc1.split()):
    one_hot_vectors[i,vocabulario.index(w)]=1
one_hot_vectors=one_hot_vectors.astype(int)
print(one_hot_vectors) # ya tenemos un vector

[[1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


In [46]:
## Podemos ver la misma información en un dataFrame para hacerlo un poco mas legible
df = pd.DataFrame(one_hot_vectors, columns=vocabulario, index=doc1.split())
df[df == 0] = '' #remplazamos los 0 con la cadena vacia
df

,Pepe,con,papas,pecas,pica,pico,un
Pepe,1,,,,,,
pecas,,,,1,,,
pica,,,,,1,,
papas,,,1,,,,
con,,1,,,,,
un,,,,,,,1
pico,,,,,,1,


para nuestro ejemplo tenemos una matriz de 7x7 ya que el vocabulario está solo constituido por una única sentencia. Recuerde que un 1 indica que el token si es parte del documento y un 0 que ese temino no se encuentra en el mismo. Este tipo de estructura es eficiente para determinar si un palabra es o no parte del un documento (solo debe verse si la fila está activa en la columna correspondiente). Además siempre es posible reconstruir el documento original, con lo cual no se pierde información. Este tipo de representación es frecuentemente utilizado en redes neuronales y modelado de lenguajes. 

La importancia de este tipo de representación es que se ha transformado una sentencia escrita en lenguaje natural, a un espacio donde es posible que una *máquina* realice operaciones matemáticas.

Una de las desventajas de tener una representación matricial de los one-hot vectors es que debido a que son altamenete disperso, su almacenamiento puede ser ineficianete si se hace de forma matricial; mientras que si lo hacemos mediante listas o alguna otra estructura dispersa se tiene un incremento en la complejida de las operaciones. 

En el caso del idioma castellano se considera que existen al rededor de unas 100,000 palabras. Teniendo lo anterior en cuenta, un one-hot vector para una palabra dada tendría que ser un vector de dimensión 10000 con único elemento diferente de 0. 
<div class="alert alert-success">
    <ul>
<li>¿Que costo tendría almacenar nuestra frase de ejemplo sin utilizar una representación dispersa que considere todas las palabras del idioma castellano? </li>
<li>¿Un documento con 100 tokens?</li>
        </ul>
</div>

In [55]:
1*20000*4*1000000

80000000000

In [6]:
# Considere que se utilizan 4 bytes por caracter o entero

### Bag of Words

Usar un representación one-hot requiere grandes cantidades de memoria, otro posible enfoque es sumarizar la información en la tabla en un solo vector. Lo anterior reduciria la memoria requerida pero se perdería la información del orden en que aparecen en el documento (de ahi el nombre de bag of words). Aún con la desventaja anterior este vector conservaria los conceptos que aparecen en el documento, sería semejante a un índice de terminos en un libro.

La sumarización puede realizarse mediante un vector binario que solo indique si la palabra aparece o no en el documento, o bien contando el número de veces que aparece cada término (vector de frecuencias)

Considere el siguiente ejemplo:

In [51]:
doc2="pecas pica papas con un pico con un pico pecas pica papas"
## Creamos la tabla de one-hot vectors 
one_hot_vectors2=np.zeros((len(doc2.split()),n)) 
## generamos la representación vectorial para nuesta frase de ejemplo
for i,w in enumerate(doc2.split()):
    one_hot_vectors2[i,vocabulario.index(w)]=1
df2 = pd.DataFrame(one_hot_vectors2, columns=vocabulario, index=doc2.split())
df2[df2 == 0] = '' #remplazamos los 0 con la cadena vacia
df2

,Pepe,con,papas,pecas,pica,pico,un
pecas,,,,1,,,
pica,,,,,1,,
papas,,,1,,,,
con,,1,,,,,
un,,,,,,,1
pico,,,,,,1,
con,,1,,,,,
un,,,,,,,1
pico,,,,,,1,
pecas,,,,1,,,


Utilizando el one-hot encoding podemos generar el vector binario como sigue:

In [52]:
np.max(one_hot_vectors2, axis=0).astype(int)

array([0, 1, 1, 1, 1, 1, 1])

también con el la tabla one-hot podemos generar el vector de frecuencias como sigue:

In [54]:
np.sum(one_hot_vectors2, axis=0).astype(int)

array([0, 2, 2, 2, 2, 2, 2])

La memoria requerida  por representación previa aún depende aún depende del tamaño del vocabulario y sigue siendo prohibitiva para grandes cantidades de datos. Una forma más ecónomica en memoria es utilizar una lista asociativa(diccionario) como  sigue:

In [56]:
#version binaria
doc1_bow = {token: 1 for token in doc1.split()}
print(doc1_bow)
doc2_bow = {token: 1 for token in doc2.split()}
print(doc2_bow)

{'Pepe': 1, 'pecas': 1, 'pica': 1, 'papas': 1, 'con': 1, 'un': 1, 'pico': 1}
{'pecas': 1, 'pica': 1, 'papas': 1, 'con': 1, 'un': 1, 'pico': 1}


In [ ]:
vocabulario=['pepe','con','papas','pecas']

In [ ]:
corpus=[[(0,1),(3,1)],[(5,1),(10,4)]]

In [58]:
#version frecuencia
doc2_bowf={}
for token in doc2.split():
   doc2_bowf[token]=doc2_bowf.get(token,0)+1
print(doc2_bowf)

{'pecas': 2, 'pica': 2, 'papas': 2, 'con': 2, 'un': 2, 'pico': 2}


In [60]:
df_bow=pd.DataFrame([doc1_bow,doc2_bow], index=['doc1','doc2']).fillna(0).astype(int)
df_bow

,Pepe,pecas,pica,papas,con,un,pico
doc1,1,1,1,1,1,1,1
doc2,0,1,1,1,1,1,1


Almacenar a la información de esta última forma es mucho más eficiente en memoria ya que cada documento ya que solo se consideran las palabras persentes en el documento.

Ahora practiquemos con un ejemplo que conste de una colección de documentos (corpus), considere el siguiente conjunto de sentencias:

In [13]:
corpus=["el rey de constantinopla esta constantinoplizado.",
        "consta que constanza no lo pudo desconstantinoplizar."
        "el desconstantinoplizador que desconstantinoplizare al rey de constantinopla",
        "buen desconstantinoplizador será"]

### Producto punto

El producto punto de dos vectores o producto escalar se calcula multiplicando todos los elementos de un vector por todos los elementos del segundo vector y luego sumando cada uno de los resultados del producto. 

En python:

In [14]:
v1 = np.array([1, 2, 3])
v2 = np.array([5, 4, 3])

In [15]:
v1.dot(v2)

22

In [62]:
np.sum(v1*v2)

22

In [64]:
sum([x1 * x2 for x1, x2 in zip(v1, v2)]) # ineficiente pero ilustrativo

22

<div class="alert alert-success">
¿Cómo podemos medir que tan similares son dos documentos?
</div>

La representación binaria de bag of words es un espacio vectorial (VSM) obtenido a partir de documentos en lenguaje natural (oraciones). En este espacio es posible realizar productos escalares, así como otras operaciones vectoriales como: suma, resta, *and*, *or*, medias, etc. También nos permite medir similitud/distancia entre documentos (i.e. distancia euclidiana,el ángulo entre vectores, etc). Como sabemos los procesadores utilizan expresiones binarias que son utilizadas para realizar indizado y realizar búsquedas de forma eficiente.

### Mejorando el tokenizador

Es frecuente, que se desee utilizar como separadores de tokens utilizan caracteres diferentes  a en una oración. Además de que nuestro rokenizador mantiene los signos de puntuación en las palabras. Una posible solución sería dividir el texto no solo en espacios en blanco, sino también en puntuación (comas, puntos, comillas, signos de amiración, etc). Sin embargo, en  algunos casos, podría desearse tratar lo signos como tokens independientes o tal vez simplemente se quiera ignorarlos.

Una de incluir diferentes patrones de división es meidante el uso de expresiones regulares.

#### Expresiones regulares

Recordemos que las expresiones sirver para expresar lenguajes regulares, y en Python se pueden utilizar mediante la librería **re**. Rvisaremos brevemente alguno aspectos del su uso:

- Los corchetes ([x]) se utilizan para indicar una tipo o un conjunto de caracteres. 
- El signo + después del corchete de cierre (]) indica que debe haber al menos una coincidencia de los caracteres dentro de los corchetes. 
- El signo * indica que cero o mas coincidencias de los caracteres dentro de la clase. 
- El símbolo  \s dentro es una clase predefinida que incluye todos los espacios en blanco como  [espacio], [tabulador]. Los seis caracteres de espacio en blanco son espacio (' '), tabulación ('\ t'), return ('\ r'), nueva línea ('\n') y  ('\f').
- Para indicar un rango de caracteres se utiliza el signo menos (-). Por ejemplo \[1-9\] indica la clase  [123456789], [a-zA-Z] hace match con los rangos de minusculas y mayusculas de los caracteres alfanuméricos. 
- Los parentesis son utilizado para agrupar expresiones regulares.
- Para expresar que se desea hacer match con -, debe ponerse justo después del corchete abierto para la clase de carácter. En caso contrario el analizador lo tomara como un rango de caracteres. 
- Los caracters especiales se pueden escapar utilizando una barra invertida.


En Python la biblioteca *re* permite compilar las expresiones regulares, con lo que se obtiene un tokenizador más eficiente. 

Las expresiones regulares también nos permiten realizar normalizaciones de texto complejas, por ejemplo extrar hyper-vínculos, direcciones de correo, nombres de usuario etc. Veremos un ejemplo sencillo de normalización y retomaremos el tema más adelante en el curso. 

In [66]:
import re
## Una expresión regular que divide utilizando signos de puntuación y espacios en blanco
patron_tokenizer=re.compile(r"([-\s.,;¿?¡!])+")

In [18]:
## Utilizaremos la siguiente frase tomada del poema Día trece de Ramón Lopez Velarde
poema="""¿En qué embriaguez bogaban tus pupilas para que así pudiesen narcotizarlo todo? 
          Tu tiniebla guiaba mis latidos, cual guiabala columna de fuego al israelita."""

In [69]:
tokens=patron_tokenizer.split(poema)
len(tokens)

51

In [70]:
print(tokens[:10]) #Los últimos 13 tokens

['', '¿', 'En', ' ', 'qué', ' ', 'embriaguez', ' ', 'bogaban', ' ']


Como podemos ver tenemos tenemos espacios en blanco, por lo que requerimos filtar los caracteres que no sean de interés.

In [71]:
no_deseados=['-',' ','\t','\n','.',';',',','¿','?','¡','!','']
tokens_sin_puntuacion=[x for x in tokens if x not in no_deseados]
print(tokens_sin_puntuacion[-12:])

['Tu', 'tiniebla', 'guiaba', 'mis', 'latidos', 'cual', 'guiabala', 'columna', 'de', 'fuego', 'al', 'israelita']


In [97]:
# hace match con todos las cadena que comienzan con @ y contienen al menos un caracter más
patron_user=re.compile(r"(@[a-zA-Z0-9\.]+)") 

In [103]:
tweet="""Hey @elon.musk  this would be so cool in synergy with what we do 
@ExoWandercraft, already letting the walking impaired walk 
autonomously http://tinyurl.com/3zaj9xqs. Hit us up! ;)"""

In [96]:
tweetu=patron_user.sub('<user>',tweet)
print(tweetu)

Hey<user>  this would be so cool in synergy with what we do 
<user>, already letting the walking impaired walk 
autonomously http://tinyurl.com/3zaj9xqs. Hit us up!


<div class="alert alert-success">
<b>EJERCICIO</b>:
Definir una expresión regular que remplace los hipervínculos con la cadena <br /> 
&lt;link &gt;
</div>

Un tokenizador puede ser tan complejo como se desee y puede querer adaptarse a una tarea especifíca. Por ejemplo en un tweet podríamos tratar de forma especial los caracteres XD. 
Existen muchas librerías en python que implementan tokenizadores especializados (dominio, idioma, etc). Las dos que mostraremos en este curso son **spaCy** y **NLTK**

In [104]:
#NLTK tokenizador de tweers 
from nltk.tokenize import TweetTokenizer
nltk_tkzr=TweetTokenizer()
nltk_tkzr.tokenize(tweet)

['Hey',
 '@elon',
 '.',
 'musk',
 'this',
 'would',
 'be',
 'so',
 'cool',
 'in',
 'synergy',
 'with',
 'what',
 'we',
 'do',
 '@ExoWandercraft',
 ',',
 'already',
 'letting',
 'the',
 'walking',
 'impaired',
 'walk',
 'autonomously',
 'http://tinyurl.com/3zaj9xqs',
 '.',
 'Hit',
 'us',
 'up',
 '!',
 ';)']

In [105]:
# !python -m spacy download en_core_web_sm #para poner el modelo inglés
#!python -m spacy download es_core_news_sm # para poner el modelo español
import spacy 
nlp = spacy.load("en_core_web_sm") # spacy Inglés 

In [106]:
doc=nlp(tweet)
for token in doc:
    print(token.text)

Hey
@elon.musk
 
this
would
be
so
cool
in
synergy
with
what
we
do


@ExoWandercraft
,
already
letting
the
walking
impaired
walk


autonomously
http://tinyurl.com/3zaj9xqs
.
Hit
us
up
!
;)


## N-Gramas

En nuestro contexto un $n$-grama es una secuencia de $n$ tokens extraidos de un documentos. Estas secuencias nos permiten incluir en el vocabulario términos que están relacionados y aparecen juntos de forma recurrente. Por ejemplo la siguiente frase:

*Nueva York es la ciudad más poblada de los Estados Unidos*

Los $n$-gramas son importantes por que nos ayudan a conservar el significaco, por ejemplo los términios **Nueva York** y **Estados Unidos** adquieren diferente significado a si son separados. Si extendemos el vocabulario mediante la inclusión del $n$-gramas nuestro sistema de procesamiento de lenguaje natural podra retener parte del contexto (order y significado) en el texto.

Como resultado del uso de $n$-gramas,  ahora se debe determinar cuales aportan de ellos la mayor cantidad de información, y así poder reducir la cantidad de $n$-gramas) incluidos en el vocabulario. Esto nos ayudará a reconocer *"Nueva York"*, sin considerar terminos  *"nueva sociedad"*. Más adelante revisaremos a detalle estrategías para identificar $n$-gramas relevantes.

El problema con utilizar $n$-gramas que habrá muchas tokens irrelevantes lo que hará que nuestro vocabulario cresca desmedidamente.  Por ejemplo de la frase anterior el $2$-grama  *"york es"* es muy porbable que no proporcione infomación relevante para nuestro sistema de PLN.  Si los  $n$-gramas son extremadamente raros, no tienen ninguna correlación con otras palabras que puedan usar para ayudar a identificar temas que conecten documentos o clases de documentos. Por lo general la mayoría de los $2$-gramas son bastante raros, más aún los de 3 y 4 tokens.

Si se utilizan $n$-gramas de forma indiscriminada la dimensionalidad del vector de características podría facilmente sobrepasar el tamaño de los documentos, lo cual sería contraproducente. Más adelane en el curso revisaremos algunas técnicas estadisticas para determinar $n$-gramas relevantes. 

También los $n$-gramas muy comunes pueden generar un incoveniente para los sistemas de PNL. Considere el bigrama "de los" de la frase anterior. Ese tipo de bigramas, seguramente apaarecen en la mayoría de sus documentos. El hecho de que se a común hace que pierda su utilidad para discriminar entre documentos, lo cual resulta en poco poder predictivo. Al igual cualquier otro token, los $n$-gramas generalmente debería omitirse si ocurren con demasiada frecuencia. Por ejemplo, si un token o $n$-grama aparece en más del 25% de los documentos del corpus, podría no incluirse como parte del vocabulario.

### Stop words (palabras vacías)
Este término se refiere a palabras comunes, es decir que ocurren con una frecuencia alta y que por lo general contienen poca información del significado de una frase. Ejemplos de algunas palabras des este tipo son:  artículos (el, las, los, un), preposiciones (sin, por, para ...) y conectores (y, o, entonces, etc).  

Tradicionalmente, es común que los sistemas de PLN excluyan las stop words del vocabulario para reducir su complejidad. Sin embargo, a pesar de que las stop words aportan poca información, estás pueden aportar información relevante cuando forman parte de $n$-gramas.

Por ejemplo en las siguientes oraciones:

- se requiere tener celular y computadora 
- se requiere tener celular o computadora

En el ejemplo previo si se utilizan tri-gramas y se remueven las stop words se tendría el token *tener celular computadora*, mientras que si no se remuven y se generan $4$-gramas se generarían los elementos  *tener celular y computadora* y *tener celular o computadora*, con lo cual se decribe mejor el siginificado de cada clase.



Remover o no las stop words depende de cada aplicación particular. Aún cuando el no remover las stop words podría reducir el vocabulario, la realidad es que estas no representan más en unos pocos cientos de palabras (al rededor de 300 para español). El omitirlas tendrá poco impacto cuando se utilizan solo $1$-gramas y aún mucho menor cuando $n>1$. Por ejemplo si asumimos que un corpus de utilizan 20,000 palabras el quitar las stop words nos dejará con alrededor de 19700 terminos en nuestro vocabulario. Cuando se utilizan bigramas el vocabulario resultante incluiria millones de terminos con lo que el ahorro de memoria sería aún mucho menor.  
Debido a lo anterior, si tiene suficientes recursos de memoría y procesamiento es más recomendable no remover las stop words. 

También dependiendo de que tanta información quiera conservar o descartar es posible solo elegir un subconjunto de stop words.

In [111]:
N=20000-500
n=2

In [109]:
from math import factorial as fac

In [110]:
fac(N)/(fac(n)*(fac(N-2)))

199990000.0

In [112]:
fac(N)/(fac(n)*(fac(N-2)))

190115250.0

In [113]:
##Lista de stop words nltk
import nltk
#nltk.download('stopwords') #Solo ejecutar la primera vez
stop_words = nltk.corpus.stopwords.words('spanish')

In [114]:
len(stop_words)# numero de stop words

313

In [120]:
tokens_sin_stop_words=[x for x in tokens_sin_puntuacion if x not in  stop_words]

In [122]:
len(tokens_sin_stop_words)

15

In [123]:
len(tokens_sin_puntuacion)

24

In [115]:
# Lista de stop words spacy
nlp = spacy.load("es_core_news_sm")
len(nlp.Defaults.stop_words)

551

In [125]:
len([x for x in tokens_sin_puntuacion if x not in  nlp.Defaults.stop_words])

14

## Normalización

Como ya hemos mencionado, el tamaño del vocabulario impacta en el desempeño de un sistema de PLN. Otra estrategia ampliamente utilizada para la reducción del vocabulario es la normalización de términos. Por ejemplo la sustitución del los nombres de usuario en el un tweet. Esto ayuda a conseguir que lo tokens que significan cosas similares se combinen en una única forma *normalizada*. Lo cual reduce el tamaño del vocabulario y también mejora la asociación de significados entre esas diferentes formas de un token o $n$-grama. El tener un vocabulario reducido ayuda a reducir el overfitting de los sistemas de aprendizaje. Por ahora la unica normalización que realizaremos será **case folding**

### Case folding 

Esta técnica consiste uniformizar las palabra que solo difieren en el uso de mayúsculas. Utilizamos mayúsculas es al inicio de un parráfo o después de un punto, en nombres propios o para dar énfasis, por lo que al hacer case folding podríamos estar perdiendo información. Sin  embargo, esta normalización ayuda a reducir el vocabulario y ayuda a unificar las palabras que pretenden significar lo mismo (y que se escriban igual) en una unidad del vocabulario. 

Case folding puede provocar que dos palabras con diferentes significados sean representados por el mismo token. Por ejemplo en las frases:

- *El Papa León XIII falleció a los 93 años*
- *El león es el rey de la selva*

En ambos casos la palabra *león* es un sustantivo, pero en el primer caso es un nombre propio nombre propio.




<div class="alert alert-success">
<b>EJERCICIOS</b>:<br />
    
Implemtar una clase con los siguientes métodos:
 <ul>
     <li>Método fit(corpus) </li>
<li>tokenizer(corpus, ngrmas=[],case_folding=True,user_replace=True,...))que incluya las siguientes caraterísticas:</li>
    <ul>
    <li>Dividir los tokens por signos de puntuación y espacios</li>
    <li>Especificar mediante una lista los diferentes tamaños de n-gramas (por ejemplo si ngram=[2,4]  generará tokens de tamaño 1, 2 y 4). Simpre genera los 1-grama.</li>
      <li>Normalizaciones (variables boleanas True/False)</li>
        <ul>
            <li>Case folding </li>
            <li>Remplazo de nombre de usuario @user-&gt;&lt;user&gt;)</li>
            <li>Remplazo de nombre de números 3424-&gt;&lt;number&gt;</li>
            <li>Remplazo de nombre hiper-vinculos http://XXX-&gt;&lt;link&gt;</li>
        </ul>
    </ul><br />
     <li>Una función transform(corpus) que reciba un corpus y regrese un lista con la representación de bolsa de palabras para cada documento</li>
    </ul>
</div>

In [34]:
class Tokenizador:
    def __init__(self,corpus=[],ngrams=[2],case_folding=True):
        self.vocabulario={"token": count}
        if corpus:
            self.vocabulario=_construye_vocabulario(corpus)
            
    def _construye_vocabulario(corpus):
        ## Aplicar normalizaciones al corpus
        ## construir tabla de vocabulario
        pass
    
    def fit(self, corpus):
           self.vocabulario=_construye_vocabulario(corpus) 
            
    def tokenizador(self, corpus):
        ## Aplicar normalizaciones al texto
        ## Tokenizar el texto utilizando el vocabulario 
        #(decidir que hacer cuando ocurren palabras que no se encuentres en el vocabulario)
        ## regresar lista de tokens
        pass
    
    def transform(self, corpus):
        ## tokenizar corpus
        ## generar vectores 
        ## regresar dataframe (solo ilustrativo, nos quedaremos con la representación en forma de dict)
        pass
        
        

In [129]:
corpus=["el rey de constantinopla esta constantinoplizado.",
        "consta que constanza no lo pudo desconstantinoplizar.",
        "el desconstantinoplizador que desconstantinoplizare al rey de constantinopla",
        "buen desconstantinoplizador será"]

In [131]:
vocabulario={}
for i,doc in enumerate(corpus):
    #aplicar normalizacion
    for word in doc.split():
        vocabulario[word]=vocabulario.get(word,0)+1

In [133]:
vocabulario

{'el': 2,
 'rey': 2,
 'de': 2,
 'constantinopla': 2,
 'esta': 1,
 'constantinoplizado.': 1,
 'consta': 1,
 'que': 2,
 'constanza': 1,
 'no': 1,
 'lo': 1,
 'pudo': 1,
 'desconstantinoplizar.': 1,
 'desconstantinoplizador': 2,
 'desconstantinoplizare': 1,
 'al': 1,
 'buen': 1,
 'será': 1}

In [136]:
doc_tokenizados=[]
for i,doc in enumerate(corpus):
    #aplicar normalizacion
    tokens={}
    for word in doc.split():
        tokens[word]=1
    doc_tokenizados.append(tokens)
        
        

In [137]:
pd.DataFramedoc_tokenizados

[{'el': 1,
  'rey': 1,
  'de': 1,
  'constantinopla': 1,
  'esta': 1,
  'constantinoplizado.': 1},
 {'consta': 1,
  'que': 1,
  'constanza': 1,
  'no': 1,
  'lo': 1,
  'pudo': 1,
  'desconstantinoplizar.': 1},
 {'el': 1,
  'desconstantinoplizador': 1,
  'que': 1,
  'desconstantinoplizare': 1,
  'al': 1,
  'rey': 1,
  'de': 1,
  'constantinopla': 1},
 {'buen': 1, 'desconstantinoplizador': 1, 'será': 1}]